An example for Tensor Decomposition

In [4]:
%load_ext autoreload
%autoreload 2
import cbx as cbx
from cbx.dynamics import CBO
from cbx.objectives import Rastrigin
import cbx.utils.termination as term
from cbx.scheduler import effective_sample_size


import numpy as np
import tensorly as tl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define the tensor that we want to decompose

In [15]:
# Dimensions of the 3-way tensor
I, J, K = 10, 12, 14

# Create a random tensor
tensor = tl.tensor(np.random.random((I, J, K)))

# Define rank
rank = 5

In [16]:
tensor

array([[[3.47542473e-01, 1.61587180e-01, 8.36784908e-01, ...,
         2.89951572e-02, 1.49090925e-01, 5.73851234e-01],
        [9.06625824e-01, 5.94615378e-01, 4.30650923e-01, ...,
         2.95290275e-01, 9.53883857e-01, 4.84145337e-01],
        [3.33627397e-01, 4.04106598e-01, 2.59738389e-01, ...,
         8.41382826e-01, 7.01582390e-01, 6.06459010e-01],
        ...,
        [7.60288918e-01, 1.03274570e-01, 8.54421583e-01, ...,
         4.11912934e-01, 9.37773011e-01, 1.34653308e-01],
        [3.73790188e-01, 2.71302096e-02, 7.36551744e-01, ...,
         3.65314103e-01, 3.18341186e-01, 3.13748651e-01],
        [9.26858263e-01, 6.02257333e-01, 4.19094063e-01, ...,
         5.98749192e-01, 6.89607817e-01, 6.27662357e-01]],

       [[2.88287236e-01, 1.44683425e-01, 7.56121454e-01, ...,
         3.13644631e-01, 9.19153196e-01, 1.43374172e-01],
        [2.37565620e-01, 1.56446392e-01, 9.77914814e-01, ...,
         5.48831315e-01, 6.20184544e-01, 2.09695383e-01],
        [3.93602039e-01, 

Define the parameters

In [17]:
conf = {'alpha': 40.0,
        'dt': 0.1,
        'sigma': 1.,
        'lamda': 1.0,
        'batch_args':{
        'batch_size':200,
        'batch_partial': False},
        'I': tensor.shape[0],
        'J': tensor.shape[1],
        'K': tensor.shape[2],
        'rank': rank,
        'd': 3,
        'term_criteria':[term.max_it_term(50)],
        'N': 100, # number of the particles
        'M': 100, # iterations
        'track_args': {'names':
                       ['update_norm',
                        'energy','x',
                        'consensus',
                        'drift']},}

Define the initial positions of the particles

In [26]:
particles = cbx.utils.init_particles_as_multiple_matrices(N=conf['N'], M=conf['M'], I=conf['I'], J=conf['J'], K=conf['K'], rank=conf['rank'], x_min=-1.0, x_max=1.0, delta=1.0, method="uniform")

In [28]:
particles[0]

array([[[ 0.53552564, -0.67781165, -0.44869656, -0.58030495,
          0.28355114],
        [-0.13223797, -0.63266974,  0.21146398, -0.90593343,
         -0.85884292],
        [-0.53760955,  0.42154118, -0.69173564,  0.75968865,
          0.80574545],
        [-0.98352711, -0.44598677, -0.29826268, -0.96343132,
         -0.75782789],
        [ 0.86597259, -0.60231481,  0.4722938 ,  0.0430683 ,
         -0.36980046],
        [ 0.19454449, -0.23275745, -0.48030783, -0.00283925,
          0.79143598],
        [-0.12980923, -0.98627511,  0.01250175,  0.68178861,
          0.91701768],
        [ 0.21355004, -0.01915389, -0.80369672,  0.98706907,
          0.04294182],
        [ 0.17827787,  0.91829268, -0.86831147,  0.91689281,
          0.57694171],
        [-0.80755461, -0.98975692, -0.75359856,  0.03606716,
         -0.52897311],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.   

In [ ]:
def objective_function(particle):
    rank = 5
    # Reconstruct tensor from the given particle
    A = particle['A']
    B = particle['B']
    C = particle['C']
    reconstructed_tensor = cp_to_tensor((np.ones(rank), [A, B, C]))

    # Compute the Frobenius norm of the difference
    error = tl.norm(tensor - reconstructed_tensor)

    return error